# Chapter 10: Introduction on Artificial Neural Network

This chapter introduces how to build an Artificial Neural Network using TensorFlow, both with higher level APIs and lower level operators. It is much easier to create a neural network with Keras, but it will be necessary to know how to use lower level operators to write your own function when needed.

> This jupyter notebook contains samples codes on the book. I personally retype them here to make me familiar with coding using TensorFlow.

## Using higher level API to train MLP

TensorFlow itself contains higher level APIs to help you build your model much easier and faster. You only need to specify the parameters, which represents the structure of your model.

In [ ]:
import tensorflow as tf
feature_columns = tf.contrib.learn.infer_real_valued_columns_from_input(X_train)
dnn_clf = tf.contrib.learn.DNNClassifier(hidden_units=[300, 100], n_classes=10, feature_columns=feature_columns)
dnn_clf.fit(x=X_train, y=y_train, batch_size=50, steps=40000)

The code above is quite similar to what you will write with Scikit-Learn. You first create a model with a function, whose parameters specify the structure of your model. Then you call its ```fit``` method to fit your training data. The only difference is that you have to specify batch size and epochs of the training process in TensorFlow, while in Sklearn you don't.

## Using pure TensorFlow to train DNN

If you want more control on your model, you can use lower level operators of TensorFlow to create your own model, namely pure TensorFlow.

### Construction Stage

In [ ]:
import tensorflow as tf
import numpy as np

# hyperparameters
n_inputs = 28 * 28    # MNIST
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        
        stddev = 2 / np.sqrt(n_inputs)
        init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)
        W = tf.Variable(init, name="weights")
        b = tf.Variable(tf.zeros([n_neurons]), name="biases")
        z = tf.matmul(X, W) + b
        if activation == "relu":
            return tf.nn.relu(z)
        else:
            return z
        
with tf.name_scope("dnn"):
    hidden1 = neuron_layer(X, n_hidden1, "hidden_1", activation="relu")
    hidden2 = neuron_layer(hidden1, n_hidden2, "hidden_2", activation="relu")
    logits = neuron_layer(hidden2, n_outputs, "outputs")

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")
    
learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
init = tf.global_variables_initializer()
saver = tf.train.Saver()

### Execution Stage

In [5]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")

n_epochs = 400
batch_size = 50

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: mnist.test.images, y: mnist.test.labels})
        print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)
        
    save_path = saver.save(sess, "./my_model_final.ckpt")

(60000, 28, 28)

### Using the trained NN

In [ ]:
with tf.Session() as sess:
    saver.restore(sess, "./my_model_final.ckpt")
    X_new_scaled = [...]
    Z = logits.eval(feed_dict={X: X_new_scaled})
    y_pred = np.argmax(Z, axis=1)